In [59]:
import os
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [60]:
# ---- PyTorch: Load model, one-hot encoder, label encoder, and scaler ----
import torch
import torch.nn as nn
import pickle

In [61]:
device=torch.device("cuda" if torch.cuda.is_available() else 'cpu')

# 1) Load the encoder(s) and scaler (same as TF, pickle works unchanged)
with open (r'onehot_encoder_geo.pkl','rb') as f:
    label_encoder_geo=pickle.load(f)
with open (r'label_encoder_gender.pkl','rb') as f:
    label_encoder_gender=pickle.load(f)
with open (r'scaler.pkl','rb') as f:
    scaler=pickle.load(f)



In [62]:
# 2) Recreate the ANN architecture EXACTLY as trained
class ANN_Model(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net =nn.Sequential(
            nn.Linear(input_dim,64), #HL1
            nn.ReLU(),
            nn.Linear(64,32), #HL2
            nn.ReLU(),
            nn.Linear(32,1), #Output layer
            nn.Sigmoid(),   #Binary Classification
        )
    def forward(self,x):
        return self.net(x)

In [63]:
# 3) Instantiate and load trained weights
input_dim= getattr(scaler,"n_features_in_", None)
if input_dim is None:
    raise ValueError("Set input_dim to the feature count used during training.")
model =ANN_Model(input_dim=input_dim).to(device)

# Replace the filename with whatever you saved (e.g., 'model.pth' or 'best_weights.pth')
state_dict=torch.load(r'C:\Users\E40056416\Documents\Extra\learning\AIML\GenAI\10_Section_13\model.pth', map_location=device)
model.load_state_dict(state_dict)
model.eval()

C:\Users\E40056416\AppData\Local\Temp\ipykernel_30396\4240033109.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict=torch.load(r'C:\Users\E40056416\Documents\Ext

ANN_Model(
  (net): Sequential(
    (0): Linear(in_features=12, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [64]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [65]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


c:\Users\E40056416\AppData\Local\anaconda3\envs\AI_ML\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [66]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [67]:
# Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df["Gender"])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [68]:
# Concat with one hot encoded
input_df=pd.concat([input_df.drop('Geography', axis=1),geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [69]:
# scalling the input data
input_scaled= scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [70]:
# --- Predict churn (PyTorch) ---
import numpy as np
import torch
model.eval()

# make sure it's float32 and on the right device
x= torch.from_numpy(input_scaled.astype(np.float32)).to(device)
with torch.no_grad():
    probs=model(x).squeeze(1).cpu().numpy() # shape: (N,)
    preds=(probs>=0.5).astype(int)  # 1 = churn, 0 = not-churn

print('probability:', float(probs[0]))
print('prediction:', int(preds[0]))




probability: 0.03623414784669876
prediction: 0
